now can output the trace for pi but have to adjust the values for pi

In [ ]:
import numpy as np
import time
!pip3 install pyro-ppl
import pyro
from pyro.distributions import *
from pyro.infer import Predictive, SVI, Trace_ELBO
from pyro.optim import Adam
from pyro.nn import PyroParam

import torch
import torch.nn as nn

clust1 = np.random.normal(0, 1, 1000)
clust2 = np.random.normal(12, 1, 1000)
clust3 = np.random.normal(6, 1, 1000)
clust4 = np.random.normal(3, 1, 1000)
clust5 = np.random.normal(9, 1, 1000)

data = torch.from_numpy(np.concatenate((clust1, clust2, clust3, clust4, clust5))).double()
N = data.shape[0]

T = 5  # Fixed number of components.

class MixturePropsMLP(nn.Module):
  '''
    Outputs a probability vector of length T
  '''
  def __init__(self):
    super().__init__()
    hidden_layer_1_size = 128
    hidden_layer_2_size = 128
    output_size = T
    self.layers = nn.Sequential(
        nn.AdaptiveAvgPool1d(1),
        nn.Linear(1, hidden_layer_1_size),
        nn.ReLU(),
        nn.Linear(hidden_layer_1_size, hidden_layer_2_size),
        nn.ReLU(),
        nn.Linear(hidden_layer_2_size, output_size),
        nn.Softmax(0)  # 0-dim because only 1D
    )

  def forward(self, x):
    return self.layers(x.unsqueeze(0).unsqueeze(0))

class MixtureParamsMLP(nn.Module):
  '''
    Outputs a probability vector of length T
  '''
  def __init__(self):
    super().__init__()
    hidden_layer_1_size = 128
    hidden_layer_2_size = 128
    output_size = T
    self.layers = nn.Sequential(
        nn.AdaptiveAvgPool1d(1),
        nn.Linear(1, hidden_layer_1_size),
        nn.ReLU(),
        nn.Linear(hidden_layer_1_size, hidden_layer_2_size),
        nn.ReLU(),
        nn.Linear(hidden_layer_2_size, output_size)
    )

  def forward(self, x):
    return self.layers(x.unsqueeze(0).unsqueeze(0))


In [ ]:
def model(data):
    with pyro.plate('components', T):
        locs = pyro.sample('locs', Normal(0, 1))

    with pyro.plate('data', N):
        # Local variables.
        assignments = pyro.sample('assignments', Categorical(torch.ones(T) / T)) # returns a vector of length T
        obs = pyro.sample('obs', Normal(locs[assignments], 1), obs=data)

def guide(data):
    # amortize using MLP
    pyro.module('pi_mlp', pi_mlp)
    
    # sample mixture components mu
    tau = pyro.param('tau', lambda: Normal(0, 1).sample([T]))
    with pyro.plate('components', T) as i:      
        pyro.sample('locs', Normal(tau[i], 1))
    
    # sample cluster assignments
    pi = pi_mlp(data.double()) # returns a vector of length T

    # pyro.param('phi', phi)    
    with pyro.plate("data", N):
        pyro.sample("assignments", Categorical(pi)) # returns a vector of length 

def print_progress(step):
    print('='*10, 'Iteration {}'.format(step), '='*10)
    tau = pyro.param('tau')

    pi_mlp = MixturePropsMLP().double()
    pi_mlp.pi = PyroParam(torch.randn(T).exp(), constraint = constraints.interval(0, 1))

    print('tau is', tau)
    print('pi is', pi_mlp.pi.init_value / torch.sum(pi_mlp.pi.init_value))

# pi_mlp = MixturePropsMLP().double()
adam_params = {"lr": 0.005}
optimizer = Adam(adam_params)
svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

n_steps = 2501
start = time.time()
for step in range(n_steps):
  svi.step(data)
  pyro.get_param_store()
  if step % 100 == 0:
        end = time.time()
        print_progress(step)
        print('took', end-start, 'seconds')
        start = time.time()